<a href="https://colab.research.google.com/github/Snow-Wang999/Developer-Books/blob/master/Parallel%20and%20High%20Performance%20Computing/dazuoye_SparkMLlib_url_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.1Install sparkNLP








In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-06-28 15:38:31--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-28 15:38:31--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.3 and Spark NLP 3.1.1
-                   100%[==

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 3.1.1
Apache Spark version: 3.0.3


In [ ]:
#!pip install elephas

##1.2 Operate environment configuration

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
%matplotlib inline 
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax

from pyspark.sql import Row
from pyspark.sql.functions import lower, col, udf
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import mean, min, max

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import PCA
from pyspark.ml.classification import NaiveBayes, LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline 

In [ ]:
'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from elephas.spark_model import SparkModel
'''

'\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D\nfrom tensorflow.keras.optimizers import SGD\nfrom keras.wrappers.scikit_learn import KerasClassifier\nfrom elephas.spark_model import SparkModel\n'

In [ ]:
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.linalg import SparseVector, DenseVector
#from pyspark.mllib.linalg.distributed import RowMatrix
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.classification import LogisticRegressionWithSGD, SVMWithSGD, SVMModel, NaiveBayes, NaiveBayesModel

In [ ]:
#from six.moves import zip

## 1.3 upload files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_2 = spark.read.csv('/content/drive/MyDrive/archive/dmoz.csv',inferSchema=True, header=True,)
df_2 = df_2.withColumnRenamed("desc","text")
df_2 = df_2.withColumn('text', lower(col('text')))\
      .withColumn('title', lower(col('title')))
df_2.show(truncate=False)

+---+--------+----------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|category|title                                                     |text                                                                                                                                                                                                                         |
+---+--------+----------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |Arts    |about.com: animation guide                                |keep up with developments in online ani

In [ ]:
df_2.select([count(when(isnull(c), c)).alias(c) for c in df_2.columns]).show()

+---+--------+-----+----+
|_c0|category|title|text|
+---+--------+-----+----+
|  0|       0|    0|   0|
+---+--------+-----+----+



In [ ]:
df_2.count()

1195851

In [ ]:
df_2.groupby("Category").count()\
    .orderBy("count",ascending=False)\
    .show()

+----------+------+
|  Category| count|
+----------+------+
|  Business|204910|
|   Society|195880|
|      Arts|193914|
|   Science| 96766|
| Computers| 93204|
|    Sports| 85376|
|Recreation| 83618|
|  Shopping| 78184|
|    Health| 50388|
| Reference| 47428|
|     Games| 36454|
|      Home| 22241|
|      News|  7488|
+----------+------+



In [ ]:
df = df_2.select("Category","title","text")
df.show(5)

+--------+--------------------+--------------------+
|Category|               title|                text|
+--------+--------------------+--------------------+
|    Arts|about.com: animat...|keep up with deve...|
|    Arts|           toonhound|british cartoon, ...|
|    Arts|enculturation: fr...|essay by patricia...|
|    Arts|digital media fx:...|michael crandol t...|
|    Arts|spark online: onl...|chris romano feel...|
+--------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
selected_rows = df.groupby("Category")\
    .count()\
    .orderBy("count",ascending=False)\
    .limit(5)\
    .collect()
selected_topics = [row.Category for row in selected_rows]
selected_topics

['Business', 'Society', 'Arts', 'Science', 'Computers']

In [ ]:
df = df.filter(F.col("Category").isin(selected_topics))
df.count()

784674

In [ ]:
df.groupby("Category").count()\
    .orderBy("count",ascending=False)\
    .show()

+---------+------+
| Category| count|
+---------+------+
| Business|204910|
|  Society|195880|
|     Arts|193914|
|  Science| 96766|
|Computers| 93204|
+---------+------+



In [ ]:
df_c1 = df.filter(F.col('Category')=='Business').limit(10000)
df_c2 = df.filter(F.col('Category')=='Society').limit(10000)
df_c3 = df.filter(F.col('Category')=='Arts').limit(10000)
df_c4 = df.filter(F.col('Category')=='Science').limit(10000)
df_c5 = df.filter(F.col('Category')=='Computers').limit(10000)
df = df_c1.union(df_c2)\
      .union(df_c3)\
      .union(df_c4)\
      .union(df_c5)
df.count()

50000

In [ ]:
df.groupby("Category").count()\
    .orderBy("count",ascending=False)\
    .show()

+---------+-----+
| Category|count|
+---------+-----+
|Computers|10000|
|  Science|10000|
|  Society|10000|
|     Arts|10000|
| Business|10000|
+---------+-----+



# 3.split data

## 3.1 random split

In [ ]:
df_train, df_test, df_validation = df.randomSplit([0.7, 0.15, 0.15], seed=2021)
print(df_train.count())
print(df_test.count())
print(df_validation.count())

34868
7568
7564


## 1.4 split text,title and category

In [ ]:
df_train_x = df_train.select("title","text")
df_train_y = df_train.select("Category")
df_test_x = df_test.select("title","text")
df_test_y = df_test.select("Category")

# 2. preprocessing(pipeline)

## 2.1 pipeline ml

In [ ]:
#regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
#stopwordList = ["http","www","com","html","htm"]
#remover = StopWordsRemover(inputCol="words", outputCol="filtered",stopWords=stopwordList)
#word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol='words', outputCol='features')
#ngram = NGram(n=1, inputCol="filtered", outputCol="ngrams")
#cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=20, minDF=1.0)
#hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=20)
#cv = CountVectorizer(inputCol="ngrams", outputCol="rawFeatures", vocabSize=20, minDF=1.0)
#idf = IDF(inputCol="rawFeatures", outputCol="features")
#pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
#indexer = StringIndexer(inputCol="Category", outputCol="label")
#lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
#nb = NaiveBayes(featuresCol="features", labelCol="label", smoothing=111, modelType="multinomial")
#pipeline = Pipeline(stages=[regexTokenizer,  word2Vec, indexer])

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
ngram_1 = NGram(n=1, inputCol="filtered", outputCol="ngram_1")
ngram_2 = NGram(n=2, inputCol="filtered", outputCol="ngram_2")
hashingTF_1 = HashingTF(inputCol="ngram_1", outputCol="rawFeatures_1", numFeatures=150)
hashingTF_2 = HashingTF(inputCol="ngram_2", outputCol="rawFeatures_2", numFeatures=150)
cv_1 = CountVectorizer(inputCol="ngram_1", outputCol="rawFeatures_3", vocabSize=150, minDF=2.0)
cv_2 = CountVectorizer(inputCol="ngram_2", outputCol="rawFeatures_4", vocabSize=150, minDF=2.0)
idf_hash_1 = IDF(inputCol="rawFeatures_1", outputCol="features_1")
idf_hash_2 = IDF(inputCol="rawFeatures_2", outputCol="features_2")
idf_cv_1 = IDF(inputCol="rawFeatures_3", outputCol="features_3")
idf_cv_2 = IDF(inputCol="rawFeatures_4", outputCol="features_4")
pca_1 = PCA(k=3, inputCol="features_1", outputCol="pcaFeatures_1")
pca_2 = PCA(k=3, inputCol="features_2", outputCol="pcaFeatures_2")

indexer = StringIndexer(inputCol="Category", outputCol="label")
nb = NaiveBayes(featuresCol="features", labelCol="label", smoothing=1, modelType="multinomial")
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100, regParam=0.3, elasticNetParam=0.8)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees = 100, maxDepth = 4, maxBins = 32)

In [ ]:
asseminputCols = ["features_1","features_2"]
assembler = VectorAssembler(inputCols=asseminputCols, outputCol="features")
pipeline = Pipeline(stages=[regexTokenizer, remover, ngram_1, ngram_2, hashingTF_1, hashingTF_2, idf_hash_1, idf_hash_2, assembler, indexer, nb])

In [ ]:
pipeline_2 = Pipeline(stages=[regexTokenizer, remover, ngram_1, ngram_2, hashingTF_1, hashingTF_2, idf_hash_1, idf_hash_2, assembler, indexer, lr])
pipeline_3 = Pipeline(stages=[regexTokenizer, remover, ngram_1, ngram_2, hashingTF_1, hashingTF_2, idf_hash_1, idf_hash_2, assembler, indexer, rf])

## 2.2 pipeline sparknlp

preprocess text data

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

In [ ]:
sentence_detector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

In [ ]:
regexTokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

In [ ]:
stop_words_cleaner = StopWordsCleaner\
    .pretrained('stopwords_en', 'en')\
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


In [ ]:
lemmatizer = LemmatizerModel\
    .pretrained("lemma_antbnc", "en") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [ ]:
ngram_1 = NGram(n=1, inputCol="token_features", outputCol="ngram_1")
ngram_2 = NGram(n=2, inputCol="token_features", outputCol="ngram_2")
hashingTF_1 = HashingTF(inputCol="ngram_1", outputCol="rawFeatures_1", numFeatures=150)
hashingTF_2 = HashingTF(inputCol="ngram_2", outputCol="rawFeatures_2", numFeatures=150)
idf_hash_1 = IDF(inputCol="rawFeatures_1", outputCol="features_1")
idf_hash_2 = IDF(inputCol="rawFeatures_2", outputCol="features_2")

preprocess title data

In [ ]:
document_assembler_2 = DocumentAssembler()\
    .setInputCol("title")\
    .setOutputCol("document_2")

In [ ]:
regexTokenizer_2 = Tokenizer() \
    .setInputCols(["document_2"]) \
    .setOutputCol("token_2")

In [ ]:
finisher_2 = Finisher() \
    .setInputCols(["token_2"]) \
    .setOutputCols(["token_features_2"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [ ]:
ngram_3 = NGram(n=1, inputCol="token_features_2", outputCol="ngram_3")
ngram_4 = NGram(n=2, inputCol="token_features_2", outputCol="ngram_4")
hashingTF_3 = HashingTF(inputCol="ngram_3", outputCol="rawFeatures_3", numFeatures=150)
hashingTF_4 = HashingTF(inputCol="ngram_4", outputCol="rawFeatures_4", numFeatures=150)
idf_hash_3 = IDF(inputCol="rawFeatures_3", outputCol="features_3")
idf_hash_4 = IDF(inputCol="rawFeatures_4", outputCol="features_4")

In [ ]:
asseminputCols = ["features_1",'features_2',"features_3",'features_4']
assembler = VectorAssembler(inputCols=asseminputCols, outputCol="features")
indexer = StringIndexer(inputCol="Category", outputCol="label")

In [ ]:
nb = NaiveBayes(featuresCol="features", labelCol="label", smoothing=0.001, modelType="multinomial")
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100, regParam=0.3, elasticNetParam=0.8)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees = 100, maxDepth = 4, maxBins = 32)

In [ ]:
pipeline_nb = Pipeline(
            stages=[
                document_assembler,
                sentence_detector, 
                regexTokenizer,
                stop_words_cleaner,
                lemmatizer,
                finisher, 
                ngram_1, 
                ngram_2, 
                hashingTF_1, 
                hashingTF_2, 
                idf_hash_1, 
                idf_hash_2, 
                document_assembler_2,
                regexTokenizer_2,
                finisher_2,
                ngram_3, 
                ngram_4, 
                hashingTF_3, 
                hashingTF_4, 
                idf_hash_3, 
                idf_hash_4,
                assembler, 
                indexer,
                nb
                  ])

In [ ]:
pipeline_lr = Pipeline(
            stages=[
                document_assembler,
                sentence_detector, 
                regexTokenizer,
                stop_words_cleaner,
                lemmatizer,
                finisher, 
                ngram_1, 
                ngram_2, 
                hashingTF_1, 
                hashingTF_2, 
                idf_hash_1, 
                idf_hash_2, 
                document_assembler_2,
                regexTokenizer_2,
                finisher_2,
                ngram_3, 
                ngram_4, 
                hashingTF_3, 
                hashingTF_4, 
                idf_hash_3, 
                idf_hash_4,
                assembler, 
                indexer,
                lr
                  ])

In [ ]:
pipeline_rf = Pipeline(
            stages=[
                document_assembler,
                sentence_detector, 
                regexTokenizer,
                stop_words_cleaner,
                lemmatizer,
                finisher, 
                ngram_1, 
                ngram_2, 
                hashingTF_1, 
                hashingTF_2, 
                idf_hash_1, 
                idf_hash_2, 
                document_assembler_2,
                regexTokenizer_2,
                finisher_2,
                ngram_3, 
                ngram_4, 
                hashingTF_3, 
                hashingTF_4, 
                idf_hash_3, 
                idf_hash_4,
                assembler, 
                indexer,
                rf
                  ])

## 2.3 keras pipeline

# 3. classification

## 3.1 spark ml classification

In [ ]:
pipelineFit_nb = pipeline.fit(df_train)

In [ ]:
pipelineFit_nb = nb.fit(pipelined)
pipelineFit_lr = lr.fit(pipelined)
pipelineFit_rf = rf.fit(pipelined)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
df_bayes = pipelineFit_nb \
    .transform(df_test) \
    .select("Category", "label", "prediction", "sentense")
df_bayes_pandas = df_bayes.toPandas()

In [ ]:
labels_df = df_bayes\
    .select("label", "Category")\
    .distinct()\
    .orderBy("label")
labels_df.show()

+-----+---------+
|label| Category|
+-----+---------+
|  0.0|  Science|
|  1.0| Business|
|  2.0|  Society|
|  3.0|Computers|
|  4.0|     Arts|
+-----+---------+



In [ ]:
labels_raw = labels_df.collect()
labels = [row.Category for row in labels_raw]
labels

['Science', 'Business', 'Society', 'Computers', 'Arts']

In [ ]:
print(classification_report(df_bayes_pandas.label, df_bayes_pandas.prediction, target_names=labels))

              precision    recall  f1-score   support

     Science       0.53      0.55      0.54      4523
    Business       0.50      0.51      0.51      4518
     Society       0.42      0.38      0.40      4419
   Computers       0.46      0.47      0.47      4501
        Arts       0.48      0.49      0.48      4638

    accuracy                           0.48     22599
   macro avg       0.48      0.48      0.48     22599
weighted avg       0.48      0.48      0.48     22599



## 3.2 sparknlp ml naive bayes

In [ ]:
%%time
pipelineFit_nb = pipeline_nb.fit(df_train)

CPU times: user 1.24 s, sys: 141 ms, total: 1.39 s
Wall time: 3min 11s


In [ ]:
df_bayes = pipelineFit_nb \
    .transform(df_test) 
df_bayes.show(5)
#df_bayes.show(truncate=False)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|Category|               title|                text|            document|            sentence|               token|         cleanTokens|               lemma|      token_features|             ngram_1|             ngram_2|       rawFeatures_1|       rawFeatures_2|          features_1|          features_2|          document_2|             token_2|    token_features_2|             ngram_3|             ngram_4|       rawFeature

In [ ]:
df_bayes.printSchema()

root
 |-- Category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |  

In [ ]:
df_bayes_1 = df_bayes.select("Category", "label", "prediction", "text")
df_bayes_pandas = df_bayes_1.toPandas()

In [ ]:
labels_df = df_bayes\
    .select("label", "Category")\
    .distinct()\
    .orderBy("label")
labels_df.show()

+-----+---------+
|label| Category|
+-----+---------+
|  0.0|  Science|
|  1.0|  Society|
|  2.0| Business|
|  3.0|     Arts|
|  4.0|Computers|
+-----+---------+



In [ ]:
labels_raw = labels_df.collect()
labels = [row.Category for row in labels_raw]
labels

['Science', 'Society', 'Business', 'Arts', 'Computers']

In [ ]:
print(classification_report(df_bayes_pandas.label, df_bayes_pandas.prediction, target_names=labels))

              precision    recall  f1-score   support

     Science       0.49      0.48      0.49      1525
     Society       0.53      0.51      0.52      1462
    Business       0.65      0.62      0.63      1533
        Arts       0.63      0.64      0.63      1531
   Computers       0.55      0.59      0.57      1517

    accuracy                           0.57      7568
   macro avg       0.57      0.57      0.57      7568
weighted avg       0.57      0.57      0.57      7568



## 3.3 sparknlp ml logistic regression

In [ ]:
%%time
pipelineFit_lr = pipeline_lr.fit(df_train)

CPU times: user 1.11 s, sys: 143 ms, total: 1.26 s
Wall time: 2min 58s


In [ ]:
df_lr = pipelineFit_lr \
    .transform(df_test) 
df_lr.show(5)
#df_lr.show(truncate=False)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|Category|               title|                text|            document|            sentence|               token|         cleanTokens|               lemma|      token_features|             ngram_1|             ngram_2|       rawFeatures_1|       rawFeatures_2|          features_1|          features_2|          document_2|             token_2|    token_features_2|             ngram_3|             ngram_4|       rawFeature

In [ ]:
df_lr.printSchema()

root
 |-- Category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |  

In [ ]:
df_lr_1 = df_lr.select("Category", "label", "prediction", "text")
df_lr_pandas = df_lr_1.toPandas()

In [ ]:
labels_df = df_lr\
    .select("label", "Category")\
    .distinct()\
    .orderBy("label")
labels_df.show()

+-----+---------+
|label| Category|
+-----+---------+
|  0.0|  Science|
|  1.0|  Society|
|  2.0| Business|
|  3.0|     Arts|
|  4.0|Computers|
+-----+---------+



In [ ]:
labels_raw = labels_df.collect()
labels = [row.Category for row in labels_raw]
labels

['Science', 'Society', 'Business', 'Arts', 'Computers']

In [ ]:
print(classification_report(df_lr_pandas.label, df_lr_pandas.prediction, target_names=labels))

              precision    recall  f1-score   support

     Science       0.20      1.00      0.34      1525
     Society       0.00      0.00      0.00      1462
    Business       0.00      0.00      0.00      1533
        Arts       0.00      0.00      0.00      1531
   Computers       0.00      0.00      0.00      1517

    accuracy                           0.20      7568
   macro avg       0.04      0.20      0.07      7568
weighted avg       0.04      0.20      0.07      7568



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3.4 sparknlp ml random forest

In [ ]:
%%time
pipelineFit_rf = pipeline_rf.fit(df_train)

CPU times: user 1.7 s, sys: 214 ms, total: 1.91 s
Wall time: 5min 4s


In [ ]:
df_rf = pipelineFit_rf \
    .transform(df_test) 
df_rf.show(5)
#df_rf.show(truncate=False)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|Category|               title|                text|            document|            sentence|               token|         cleanTokens|               lemma|      token_features|             ngram_1|             ngram_2|       rawFeatures_1|       rawFeatures_2|          features_1|          features_2|          document_2|             token_2|    token_features_2|             ngram_3|             ngram_4|       rawFeature

In [ ]:
df_rf.printSchema()

root
 |-- Category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |  

In [ ]:
df_rf_1 = df_rf.select("Category", "label", "prediction", "text")
df_rf_pandas = df_rf_1.toPandas()

In [ ]:
labels_df = df_rf\
    .select("label", "Category")\
    .distinct()\
    .orderBy("label")
labels_df.show()

+-----+---------+
|label| Category|
+-----+---------+
|  0.0|  Science|
|  1.0|  Society|
|  2.0| Business|
|  3.0|     Arts|
|  4.0|Computers|
+-----+---------+



In [ ]:
labels_raw = labels_df.collect()
labels = [row.Category for row in labels_raw]
labels

['Science', 'Society', 'Business', 'Arts', 'Computers']

In [ ]:
print(classification_report(df_rf_pandas.label, df_rf_pandas.prediction, target_names=labels))

              precision    recall  f1-score   support

     Science       0.47      0.31      0.37      1525
     Society       0.41      0.47      0.43      1462
    Business       0.53      0.58      0.55      1533
        Arts       0.50      0.67      0.57      1531
   Computers       0.53      0.41      0.46      1517

    accuracy                           0.49      7568
   macro avg       0.49      0.49      0.48      7568
weighted avg       0.49      0.49      0.48      7568



## 3.5 keras deep learning

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD())

In [ ]:
model = Sequential()
model.add(Dense(64,input_shape=(100,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
spark_model.fit(rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)

In [ ]:
x_test, y_test = ... # load test data

predictions = spark_model.predict(x_test) # perform inference
evaluation = spark_model.evaluate(x_test, y_test) # perform evaluation/scoring

In [ ]:
estimator = ElephasEstimator(model, epochs=epochs, batch_size=batch_size, frequency='batch', mode='asynchronous',
                             categorical=True, nb_classes=nb_classes)
fitted_model = estimator.fit(df)

In [ ]:
prediction = fitted_model.transform(test_df)
pnl = prediction.select("label", "prediction")
pnl.show(100)

prediction_and_label= pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = MulticlassMetrics(prediction_and_label)
print(metrics.precision())
print(metrics.recall())

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(256, input_dim=3010, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(160, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(20, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model.fit(tfidf_mat,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

In [ ]:
print("Compile model ...")
estimator = KerasClassifier(build_fn=build_model, epochs=15, batch_size=128)
estimator.fit(X, dummy_y_train)

# Predictions 
print ("Predict on test data ... ")
y_test = estimator.predict(X_test)
y_pred = lb.inverse_transform(y_test)

# Submission
print ("Generate Submission File ... ")
test_id = [doc['id'] for doc in test]
sub = pd.DataFrame({'id': test_id, 'cuisine': y_pred}, columns=['id', 'cuisine'])
sub.to_csv('nn_output.csv', index=False)


In [ ]:
from hyperopt import STATUS_OK
from hyperas.distributions import choice, uniform

def data():
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.utils import to_categorical
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    nb_classes = 10
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)
    return x_train, y_train, x_test, y_test


def model(x_train, y_train, x_test, y_test):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Activation
    from tensorflow.keras.optimizers import RMSprop

    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    rms = RMSprop()
    model.compile(loss='categorical_crossentropy', optimizer=rms)

    model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              nb_epoch=1,
              show_accuracy=True,
              verbose=2,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, show_accuracy=True, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model.to_yaml()}

In [ ]:
from elephas.hyperparam import HyperParamModel
from pyspark import SparkContext, SparkConf

# Create Spark context
conf = SparkConf().setAppName('Elephas_Hyperparameter_Optimization').setMaster('local[8]')
sc = SparkContext(conf=conf)

# Define hyper-parameter model and run optimization
hyperparam_model = HyperParamModel(sc)
hyperparam_model.minimize(model=model, data=data, max_evals=5)

In [ ]:
result = hyperparam_model.best_ensemble(nb_ensemble_models=10, model=model, data=data, max_evals=5)